## Connecting to Azure Machine Learning Service
Interactive auth is fine for testing, experimenting, debugging, but as you move to production you should think about [Service Principal authentication](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication#service-principal-authentication)

In [1]:
import azureml.core

# Check core SDK version number.
print('SDK version:', azureml.core.VERSION)

SDK version: 1.9.0


In [31]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, sep='\n')

aml-deployment-demo
aml-deployment-demo
westeurope


## Train a model using sample dataset

In [3]:
import joblib

from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge


dataset_x, dataset_y = load_diabetes(return_X_y=True)

model = Ridge().fit(dataset_x, dataset_y)

joblib.dump(model, 'sklearn_regression_model_test.pkl')

['sklearn_regression_model_test.pkl']

## Upload Datasets and Register Model
To complete the creation process, you can register your datasets with a workspace. Use the `register()` method to register datasets with your workspace in order to share them with others and reuse them across experiments in your workspace:
```
diabetes_ds = diabetes_ds.register(workspace=workspace,
                                   name='diabetes_dsdiabetes_ds',
                                   description='sklearn training data')
```

In [12]:

import numpy as np

from azureml.core import Dataset


np.savetxt('features.csv', dataset_x, delimiter=',')
np.savetxt('labels.csv', dataset_y, delimiter=',')

datastore = ws.get_default_datastore()
datastore.upload_files(files=['./features.csv', './labels.csv'],
                       target_path='sklearn_regression/',
                       overwrite=True)

input_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'sklearn_regression/features.csv')])
output_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'sklearn_regression/labels.csv')])


Uploading an estimated of 2 files
Uploading ./features.csv
Uploading ./labels.csv
Uploaded ./labels.csv, 1 files out of an estimated total of 2
Uploaded ./features.csv, 2 files out of an estimated total of 2
Uploaded 2 files


In [10]:
import sklearn

from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = Model.register(workspace=ws,
                       model_name='my-sklearn-model',                     # Name of the registered model in your workspace.
                       model_path='./sklearn_regression_model_test.pkl',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,       # Framework used to create the model.
                       model_framework_version=sklearn.__version__,       # Version of scikit-learn used to create the model.
                       sample_input_dataset=input_dataset,
                       sample_output_dataset=output_dataset,
                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                       description='Ridge regression model to predict diabetes progression.',
                       tags={'area': 'diabetes', 'type': 'regression'})

print('Name:', model.name)
print('Version:', model.version)

Registering model my-sklearn-model
Name: my-sklearn-model
Version: 1


## Deploy the model
### Register model environment

In [17]:
from azureml.core.environment import Environment
myenv = Environment.from_conda_specification(name = 'myenv',
                                             file_path = 'inferenceconfig.yml')
myenv.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20200423.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "myenv",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "dependencies": [
                "python=3.6.9",
                "scikit-lear

In [23]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

myenv = Environment.get(workspace=ws, name='myenv', version='1')
inference_config = InferenceConfig(entry_script='score.py',
                                   environment=myenv)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = 'my-sklearn-service'

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(show_output=True)

Running..........................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [28]:
import json

input_payload = json.dumps({
    'data': dataset_x[0:2].tolist(),
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)

[182.67357342863968, 90.99902728640282]


In [27]:
# print(Environment.list(workspace=ws))

In [32]:
#dataset_x[0:2].tolist()